In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from toolbox import wrappers
from toolbox import network_utilities
import repos_tools
import pandas as pd
import numpy as np
import scipy.stats

GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!


Read network and mapping betwen gene IDs and symbols

In [2]:
id_mapping_file = '../../resources/PPI/geneid_to_symbol.txt'
network = wrappers.get_network('../../resources/PPI/Cheng2019/network.sif', only_lcc = True)
id_symbol = pd.read_csv(id_mapping_file, sep='\t', index_col='GeneID')
id_symbol = id_symbol.set_index(id_symbol.index.astype('str'))

From Andrew's manuscript on repurposing asthma drugs to AD:
> Approved drugs, or drugs in development, target the protein products of 15 of the 81 genes as identified using the ChEMBL database [28]. Among these are GRIN3A, PDE4D, and IL6R. The glutamate ionotropic NMDA receptor subunit NR3A is encoded by GRIN3A and has been previously associated with AD as targeted by an existing approved AD treatment, Memantine [29]. Phosphodiesterase-4 (PDE4) has been considered a promising target for treatment of AD and depression and functions as an existing target for current asthma medication [30], [31]. Asthma is the trait of interest in this study, thus we have focused on drugs shown to be effective in it’s treatment such as Roflumilast targeting PDE4D. Through the use of STRINGDB protein-protein interaction network we selected additional asthma drugs targeting proteins with scores >= 0.500 from those within our implicated list [27]. This included approved asthma drugs such as Salbutamol targeting beta-2 adrenergic receptor ADRB2, Montelukast targeting cysteinyl leukotriene receptor 1 CYSLTR1, and Fluticasone which targets the glucocorticoid receptor NR3C1.

In [3]:
%%bash
cat > ../../results/2021-09-02-asthma-drugs/asthma-drug-targets <<EOF
GRIN3A
PDE4D
IL6R
ADRB2
CYSLTR1
NR3C1
EOF

In [4]:
targets = wrappers.convert_to_geneid(file_name='../../results/2021-09-02-asthma-drugs/asthma-drug-targets', id_type='symbol', id_mapping_file=id_mapping_file)
d = id_symbol.loc[targets, 'Symbol'].to_dict()
D = {v: {k} for k, v in d.items()}
D

set()


{'NR3C1': {'2908'},
 'IL6R': {'3570'},
 'GRIN3A': {'116443'},
 'ADRB2': {'154'},
 'PDE4D': {'5144'},
 'CYSLTR1': {'10800'}}

In [5]:
AD_genes_knowledge = wrappers.convert_to_geneid(file_name='../../results/2021-07-01-high-conf-ADgenes/AD-genes-knowledge', id_type='symbol', id_mapping_file=id_mapping_file)
AD_genes_knowledge, AD_genes_knowledge_dropped = repos_tools.drop_genes_notin_network(AD_genes_knowledge, network)
AD_genes_HC = wrappers.convert_to_geneid(file_name='../../results/2021-07-01-high-conf-ADgenes/AD-genes-seed', id_type='symbol', id_mapping_file=id_mapping_file)
AD_genes_HC, AD_genes_HC_dropped = repos_tools.drop_genes_notin_network(AD_genes_HC, network)

{'MT-ND2', 'MT-ND1'}
{'X84075', 'MT-ND2', 'MT-ND1', 'ENSG00000260911', 'ENSG00000270081.1'}


Calculate proximity

In [6]:
def helper(target, geneset, network):
    res = wrappers.calculate_proximity(network=network, nodes_from=target, nodes_to=geneset)
    return(res)

Res_knowledge = {k: helper(v, AD_genes_knowledge, network) for k, v in D.items()}
Res_HC = {k: helper(v, AD_genes_HC, network) for k, v in D.items()}

In [8]:
Res_knowledge

{'NR3C1': (1.0, -0.6344482202508909, (1.287, 0.4523615810388853)),
 'IL6R': (1.0, -3.493482487361681, (1.94, 0.26907248094147423)),
 'GRIN3A': (2.0, 0.046301475863886325, (1.987, 0.28076858798661936)),
 'ADRB2': (2.0, 1.5800327517186143, (1.286, 0.4518893669915237)),
 'PDE4D': (1.0, -1.8297332543657676, (1.772, 0.42191942358701623)),
 'CYSLTR1': (4.0, 2.864459496157731, (2.4, 0.5585696017507578))}

In [9]:
Res_HC

{'NR3C1': (1.0, -0.2570954706397106, (1.062, 0.24115555146004825)),
 'IL6R': (1.0, -2.031798495964875, (1.805, 0.3962007067131506)),
 'GRIN3A': (2.0, 0.3346107708855587, (1.865, 0.4034538387473839)),
 'ADRB2': (1.0, -0.24813529629094064, (1.058, 0.23374344910606584)),
 'PDE4D': (1.0, -1.075040032850843, (1.544, 0.5060276672277911)),
 'CYSLTR1': (3.0, 1.420562025260845, (2.251, 0.5272561047536576))}

In [7]:
%connect_info

{
  "shell_port": 39037,
  "iopub_port": 58895,
  "stdin_port": 34813,
  "control_port": 59445,
  "hb_port": 58115,
  "ip": "127.0.0.1",
  "key": "60c6830f-5f831995eb6d215ea197c0d6",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-b8628dab-12ed-49aa-a67a-de03442afd59.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
